In [1]:
# user-friendly print
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

## Crystal structure generation

To generate reasonable crystal structure under a given space group with a specific chemical composition, basicly the following four steps are needed.

1. calculate possible Wyckoff configurations under a given space group for each chemical composition.
2. generate fraction positions for each element with given a Wyckoff configuration which is calculated from step 1), randomly.
3. generate lattice for the given space group which is used in step 1), randomly.
4. combine the results from step 2) and 3) to obtain a crystal structure.

Usually, we also have to check the `volume` and `atomic distances` of generated structure, only keep the structures which have reasonable `volume` and `atomic distances`.

To facilitate all these jobs, our `crystallus` library provides three modules:

* `WyckoffCfgGenerator`: generate possible Wyckoff configurations for the given space group and composition of primitive cell.
* `CrystalGenerator`: generate crystal structures for the given space group and Wyckoff configurations.
* `WyckoffDB, SpaceGroupDB`: database include space group and corresponding Wyckoff information.

The folloing content shows how to use `crystallus`.

### 1. Spacegroup information DB

You can use `SpaceGroupDB` to get the information of a given spacegroup such as Wyckoff positions. For example, get information of space group `167`

In [2]:
from shotgun_csp.utils import SpaceGroupDB

wys = SpaceGroupDB.get(spacegroup_num=167).wyckoffs
[
    {"Wyckoff letter": w.letter, "multiplicity": w.multiplicity, "reusable": w.reuse, "Wyckoff position": w.positions}
    for w in wys
]

[{'Wyckoff letter': 'f',
  'multiplicity': 12,
  'reusable': True,
  'Wyckoff position': '(x,y,z), (z,x,y), (y,z,x), (-y+1/2,-x+1/2,-z+1/2), (-x+1/2,-z+1/2,-y+1/2), (-z+1/2,-y+1/2,-x+1/2), (-x,-y,-z), (-z,-x,-y), (-y,-z,-x), (y+1/2,x+1/2,z+1/2), (x+1/2,z+1/2,y+1/2), (z+1/2,y+1/2,x+1/2)'},
 {'Wyckoff letter': 'e',
  'multiplicity': 6,
  'reusable': True,
  'Wyckoff position': '(x,-x+1/2,1/4), (1/4,x,-x+1/2), (-x+1/2,1/4,x), (-x,x+1/2,3/4), (3/4,-x,x+1/2), (x+1/2,3/4,-x)'},
 {'Wyckoff letter': 'd',
  'multiplicity': 6,
  'reusable': False,
  'Wyckoff position': '(1/2,0,0), (0,1/2,0), (0,0,1/2), (1/2,0,1/2), (0,1/2,1/2), (1/2,1/2,0)'},
 {'Wyckoff letter': 'c',
  'multiplicity': 4,
  'reusable': True,
  'Wyckoff position': '(x,x,x), (-x+1/2,-x+1/2,-x+1/2), (-x,-x,-x), (x+1/2,x+1/2,x+1/2)'},
 {'Wyckoff letter': 'b',
  'multiplicity': 2,
  'reusable': False,
  'Wyckoff position': '(0,0,0), (1/2,1/2,1/2)'},
 {'Wyckoff letter': 'a',
  'multiplicity': 2,
  'reusable': False,
  'Wyckoff position

### 2. generate Wyckoff configurations

As an example, we will try to generate structures for `Ca2C2O6`. The true space group of this structure is `167`, and the Wyckoff configuration is `{Ca: 2b, C: 2a, O: 6e}`.
First, let's generate some possible Wyckoff configurations for the composition `Ca2C2O6` under space group `167`.

In [3]:
from shotgun_csp.generator.wyckoff_base import WyckoffCfgGenerator

WyckoffCfgGenerator?

Init signature:
WyckoffCfgGenerator(
    composition,
    *,
    max_recurrent: int = 1000,
    n_jobs: int = -1,
    priority: Optional[Dict[int, Dict[str, float]]] = None,
    verbose: bool = False,
)
Docstring:      <no docstring>
Init docstring:
A generator for possible Wyckoff configuration generation.

Parameters
----------
max_recurrent:
    Max recurrent until generate a reasonable structure, by default 5_000
n_jobs:
    Number of cpu cores when parallel calculation, by default -1
priority:
    Priorities for Wyckoff letters. By default, a Wyckoff letter will be sampled
    from an Uniform distribution of the all available letters.
    Give this parameter will overwrite the corresponding priority list of Wyckoff letters.
    For example, space group 167 has Wyckoff letters `[a, b, c, d, e, f]`
    If priority is None, all wyckoff letters will be selected under uniform distribution.
    Now, we want to lift the priority `a`, `b` and `d`, we can give parameter `priority`
    a va

In [4]:
composition = {"Ca": 2, "C": 2, "O": 6}

wyg = WyckoffCfgGenerator(composition, verbose=False)
wyg

WyckoffCfgGenerator(            
    max_recurrent=1000,            
    n_jobs=-1            
    priority=None            
    composition={'Ca': 2, 'C': 2, 'O': 6}            
    verbose=False            
)

You have noticed that the minimum input for the initialization of a `WyckoffCfgGenerator` is just chemical composition as a python dict.
Then, we can use `wyg.gen_one` or `wyg.gen_many` methods to generate Wyckoff configuration(s).

First, let's try the `gen_one` method.

In [5]:
wyg.gen_one?

Signature: wyg.gen_one(*, spacegroup_num: int)
Docstring:
Try to generate a possible Wyckoff configuration under the given space group.

Parameters
----------
spacegroup_num:
    Space group number.

Returns
-------
Dict
    Wyckoff configuration set, which is a dict with format like:
    {"Li": ["a", "c"], "O": ["i"]}. Here, the "Li" is an available element
    symbol and ["a", "c"] is a list which contains coresponding Wyckoff
    letters. For convenience, dict will be sorted by keys.
File:      ~/projects/crystallus/shotgun_csp/generator/wyckoff_base/wyckoff_cfg_generator.py
Type:      method

In [6]:
import logging

logging.basicConfig()
logging.getLogger().setLevel(logging.INFO)

cfg = wyg.gen_one(spacegroup_num=167)
cfg

{'C': ['b'], 'Ca': ['a'], 'O': ['e']}

If everything goes well, the above cell will return a dict contains something like: `{'C': ['b'], 'Ca': ['a'], 'O': ['d']}`.
Here, `C`, `Ca`, and `O` are the element names. All elements are sorted by their alphabet. The `['b']`, `['a']`, and `['d']` are the corresponding Wyckoff positions which are provided by Wyckoff letters.

Maybe you are confused that the return of this method is not unique. That makes sense because under space group `167`, there are four possible configurations for the composition `Ca2C2O6`. Call the `gen_one` method will execute a random search in all possible configurations. When it finds one, it returns the result and stops searching. This means if you want to get more configurations, you should call the `gen_one` method many times.

We know that for almost all the cases, the possible configurations are not one, to simplify your works, we provide the `gen_many` method.

In [7]:
wyg.gen_many?

Signature: wyg.gen_many(size: int, *, spacegroup_num: Union[int, Sequence[int]])
Docstring:
Try to generate possible Wyckoff configuration sets.

Parameters
----------
size:
    How many times to try for one space group.
spacegroup_num:
    Spacegroup numbers to generate Wyckoff configurations.

Returns
-------
Dict[int, List[Dict]], List[Dict]
    A collection contains spacegroup number and it's corresponding Wyckoff
    configurations (wy_cfg). If only one spacegroup number was given,
    will only return the list of wy_cfgs, otherwise return in dict with
    spacegroup number as key. wy_cfgs will be formated as
    {element 1: [Wyckoff_letter, Wyckoff_letter, ...], element 2: [...], ...}.
File:      ~/projects/crystallus/shotgun_csp/generator/wyckoff_base/wyckoff_cfg_generator.py
Type:      method

In [8]:
cfgs = wyg.gen_many(10, spacegroup_num=167)
cfgs

[{'C': ['b'], 'Ca': ['a'], 'O': ['d']},
 {'C': ['b'], 'Ca': ['a'], 'O': ['e']},
 {'C': ['a'], 'Ca': ['b'], 'O': ['d']},
 {'C': ['a'], 'Ca': ['b'], 'O': ['e']}]

You can calculate more multiply space group in one call. Just list space group numbers as `*` parameters. In this case, the return will be a dict with space group number as key and configuration list as value. For example, if our space group candidate are `[194, 148, 167]`, you can call `gen_many` like this:

In [9]:
%%time

cfgs = wyg.gen_many(20, spacegroup_num=(194, 148, 167))
cfgs

CPU times: user 227 ms, sys: 4.27 ms, total: 231 ms
Wall time: 14.2 ms


{194: [{'C': ['c'], 'Ca': ['d'], 'O': ['b', 'f']},
  {'C': ['b'], 'Ca': ['a'], 'O': ['d', 'e']},
  {'C': ['d'], 'Ca': ['b'], 'O': ['c', 'e']},
  {'C': ['d'], 'Ca': ['b'], 'O': ['a', 'e']},
  {'C': ['d'], 'Ca': ['c'], 'O': ['b', 'e']},
  {'C': ['b'], 'Ca': ['d'], 'O': ['g']},
  {'C': ['c'], 'Ca': ['a'], 'O': ['d', 'f']},
  {'C': ['a'], 'Ca': ['d'], 'O': ['c', 'f']},
  {'C': ['a'], 'Ca': ['c'], 'O': ['g']},
  {'C': ['b'], 'Ca': ['d'], 'O': ['c', 'e']},
  {'C': ['d'], 'Ca': ['a'], 'O': ['c', 'f']},
  {'C': ['d'], 'Ca': ['a'], 'O': ['h']},
  {'C': ['d'], 'Ca': ['c'], 'O': ['b', 'f']},
  {'C': ['d'], 'Ca': ['b'], 'O': ['a', 'f']},
  {'C': ['a'], 'Ca': ['d'], 'O': ['b', 'f']},
  {'C': ['a'], 'Ca': ['c'], 'O': ['b', 'e']},
  {'C': ['b'], 'Ca': ['c'], 'O': ['g']},
  {'C': ['c'], 'Ca': ['b'], 'O': ['d', 'e']},
  {'C': ['a'], 'Ca': ['c'], 'O': ['d', 'e']},
  {'C': ['c'], 'Ca': ['a'], 'O': ['g']}],
 148: [{'C': ['c'], 'Ca': ['c'], 'O': ['f']},
  {'C': ['a', 'b'], 'Ca': ['c'], 'O': ['f']},
  {'C':

`gen_many_iter` is an iterative version of `gen_many`. You can use this method to render a progress bar during generation, or something else you want.

In [10]:
%%time

from tqdm.notebook import tqdm

space_group_cans = [194, 148, 167, 161, 11, 12, 65, 140, 225]

with tqdm(total=len(space_group_cans)) as pbar:
    for spacegroup_num, cfg_list in wyg.gen_many_iter(5000, spacegroup_num=space_group_cans):
        print(f"space group: {spacegroup_num}, size of generated samples: {len(cfg_list)}")
        pbar.update()

  0%|          | 0/9 [00:00<?, ?it/s]

space group: 194, size of generated samples: 72
space group: 148, size of generated samples: 14
space group: 167, size of generated samples: 4
space group: 161, size of generated samples: 2
space group: 11, size of generated samples: 199
space group: 12, size of generated samples: 2395
space group: 65, size of generated samples: 4358
space group: 140, size of generated samples: 96
space group: 225, size of generated samples: 4
CPU times: user 1min 52s, sys: 42.5 ms, total: 1min 52s
Wall time: 3.21 s


### 3. generate Wyckoff configurations with prior probability

Sometimes, we'd like to generate Wyckoff configurations from a given priority list of Wyckoff letters. For example, in the case of `Ca2C2O6` with space group `167`, we expect that the possible Wyckoff positions [`a`, `b`, `d`, `e`] should be sampled by the probability of [40%, 20%, 40%, 0%], respectively.
We provide the `priority` option just for these cases.

In [11]:
composition = {"Ca": 2, "C": 2, "O": 6}

wyg = WyckoffCfgGenerator(
    composition,
    priority={167: {"a": 0.4, "b": 0.2, "d": 0.4, "e": 0}},
)
wyg

WyckoffCfgGenerator(            
    max_recurrent=1000,            
    n_jobs=-1            
    priority={167: {'a': 0.4, 'b': 0.2, 'd': 0.4, 'e': 0}}            
    composition={'Ca': 2, 'C': 2, 'O': 6}            
    verbose=False            
)

When generating, the priority list will be normalized. You can see that we can only get two configurations without letter `e`.

In [12]:
cfgs = wyg.gen_many(100, spacegroup_num=167)
cfgs

[{'C': ['a'], 'Ca': ['b'], 'O': ['d']}, {'C': ['b'], 'Ca': ['a'], 'O': ['d']}]

Also you can test that if we remove letter `a` from the priority list, we can not generate any configuration because `a` and `b` must exist.

In [13]:
wyg = WyckoffCfgGenerator(
    composition,
    priority={167: {"a": 0.0}},
)
wyg.gen_many(100, spacegroup_num=167)

[]

### 4. generate crystal structures

We have generated some Wyckoff configurations, the next is consuming the Wyckoff configurations to generate crystal structures. To facilitate the task, we provide the `CrystalGenerator` class.

In [14]:
from shotgun_csp.generator.wyckoff_base import CrystalGenerator

CrystalGenerator?

Init signature:
CrystalGenerator(
    spacegroup_num: int,
    volume_of_cell: float,
    variance_of_volume: float,
    *,
    angle_range: Tuple[float, float] = (30.0, 150.0),
    angle_tolerance: float = 20.0,
    lattice: Optional[Tuple[float]] = None,
    empirical_coords: Optional[List[Tuple[str, List[float]]]] = None,
    empirical_coords_variance: float = 0.01,
    empirical_coords_sampling_rate: float = 1.0,
    empirical_coords_loose_sampling: bool = True,
    max_attempts_number: int = 5000,
    n_jobs: int = -1,
    verbose: bool = False,
)
Docstring:      <no docstring>
Init docstring:
A generator for possible crystal structure generation.

Parameters
----------
spacegroup_num:
    Specify the spacegroup.
volume_of_cell:
    The estimated volume of primitive cell. Unit is Å^3.
variance_of_volume:
    The estimated variance of volume prediction. Unit is Å^3.
    ``volume_of_cell`` and ``variance_of_volume`` will be used to build
    a Gaussion distribution for the sampling 

To initialize a `CrystalGenerator`, parameter `spacegroup_num`, `volume_of_cell` of primitive cell and the `variance_of_volume` are needed at least.

In [15]:
volume_of_cell = 127.170256
variance_of_volume = 40.0
sp_num = 167

cg = CrystalGenerator(sp_num, volume_of_cell, variance_of_volume)
cg

CrystalGenerator(            
    spacegroup_num=167,            
    volume_of_cell=127.170256,            
    variance_of_volume=40.0,            
    angle_range=(30.0, 150.0),            
    angle_tolerance=20.0,            
    max_attempts_number=5000,            
    lattice=None,            
    empirical_coords=None,            
    empirical_coords_variance=0.01,            
    empirical_coords_sampling_rate=1.0,            
    empirical_coords_loose_sampling=True,            
    verbose=False            
    n_jobs=-1            
)

Like the `WyckoffCfgGenerator`, there are also `gen_one`, `gen_many`, and `gen_may_iter` methods attached with `CrystalGenerator` object. Let's ues the `gen_one` method for a quick try.

In [16]:
cg.gen_one?

Signature:
cg.gen_one(
    wyckoff_cfg: Dict[str, Tuple[str]],
    *,
    check_distance: bool = True,
    distance_scale_factor: float = 0.1,
)
Docstring:
Try to generate a legal crystal structure with given configuration set.

Parameters
----------
wyckoff_cfg:
    Wyckoff Configuration set, which is a dict with format like:
    {"Li": ["a", "c"], "O": ["i"]}. Here, the "Li" is an available element
    symbol and ["a", "c"] is a list which contains coresponding Wyckoff
    letters. For convenience, dict will be sorted by keys.
check_distance:
    Whether the atomic distance should be checked. default ``True``
distance_scale_factor:
    Scale factor to determine the tolerance of atomic distances when distance checking. Unit is Å,
    When ``check_distance`` is ``True``, Any structure has
    all_atomic_distance < (A_atom_covalent_radius + B_atom_covalent_radius) * (1 - distance_scale_factor) will be rejected,
    by default 0.1

Returns
-------
Dict
    Structure information contains 

All `gen_xxx` methods consume Wyckoff configurations to generate crystal structures. Please note the parameter `distance_scale_factor`, generator use this parameter to determine the acceptabel atomic distance. Here is the accept condition:
> distance between atom `a` and `b` > (radius of `a` + radius of `b`) x (1 – `distance_scale_factor`).

If the generator cannot generate any structure after multiple attempts, you can try to relax the atomic distance constraint by increasing this parameter.

In [18]:
%%time

cfg = cfgs[0]
cfg

raw_s = cg.gen_one(cfg)
raw_s

CPU times: user 1.31 ms, sys: 0 ns, total: 1.31 ms
Wall time: 1.33 ms


{'spacegroup_num': 167,
 'volume': 101.48084899348507,
 'lattice': [[4.630626284667379, 0.0, 1.5123229929364803],
  [1.0970218133201437, 4.4988046111553, 1.5123229929364803],
  [0.0, 0.0, 4.871326372069189]],
 'species': ['C', 'C', 'Ca', 'Ca', 'O', 'O', 'O', 'O', 'O', 'O'],
 'wyckoff_letters': ['a', 'a', 'b', 'b', 'd', 'd', 'd', 'd', 'd', 'd'],
 'coords': [[0.25, 0.25, 0.25],
  [0.75, 0.75, 0.75],
  [0.0, 0.0, 0.0],
  [0.5, 0.5, 0.5],
  [0.5, 0.0, 0.0],
  [0.0, 0.5, 0.0],
  [0.0, 0.0, 0.5],
  [0.5, 0.0, 0.5],
  [0.0, 0.5, 0.5],
  [0.5, 0.5, 0.0]]}

The result is a dict contains `species`, `lattice`, `coords` and other information. These information can be used to build `pymatgen.Structure` or `ase.Structure` object like following.

In [19]:
# ASE Atoms

from ase import Atoms

atoms = Atoms(
    symbols=raw_s["species"],
    cell=raw_s["lattice"],
    scaled_positions=raw_s["coords"],
    pbc=True,
)
atoms

Atoms(symbols='C2Ca2O6', pbc=True, cell=[[4.630626284667379, 0.0, 1.5123229929364803], [1.0970218133201437, 4.4988046111553, 1.5123229929364803], [0.0, 0.0, 4.871326372069189]])

In [20]:
from pymatgen.core import Structure

structure = Structure(
    lattice=raw_s["lattice"],
    species=raw_s["species"],
    coords=raw_s["coords"],
)
structure

Structure Summary
Lattice
    abc : 4.871326372069189 4.871326372069189 4.871326372069189
 angles : 71.91340432634925 71.91340432634925 71.91340432634925
 volume : 101.48084899348501
      A : np.float64(4.630626284667379) np.float64(0.0) np.float64(1.5123229929364803)
      B : np.float64(1.0970218133201437) np.float64(4.4988046111553) np.float64(1.5123229929364803)
      C : np.float64(0.0) np.float64(0.0) np.float64(4.871326372069189)
    pbc : True True True
PeriodicSite: C (1.432, 1.125, 1.974) [0.25, 0.25, 0.25]
PeriodicSite: C (4.296, 3.374, 5.922) [0.75, 0.75, 0.75]
PeriodicSite: Ca (0.0, 0.0, 0.0) [0.0, 0.0, 0.0]
PeriodicSite: Ca (2.864, 2.249, 3.948) [0.5, 0.5, 0.5]
PeriodicSite: O (2.315, 0.0, 0.7562) [0.5, 0.0, 0.0]
PeriodicSite: O (0.5485, 2.249, 0.7562) [0.0, 0.5, 0.0]
PeriodicSite: O (0.0, 0.0, 2.436) [0.0, 0.0, 0.5]
PeriodicSite: O (2.315, 0.0, 3.192) [0.5, 0.0, 0.5]
PeriodicSite: O (0.5485, 2.249, 3.192) [0.0, 0.5, 0.5]
PeriodicSite: O (2.864, 2.249, 1.512) [0.5, 0.5, 

We also offer a convenient function that converts raw structural data into a `pymatgen.core.Structure` object, enriched with additional details like space group information.

In [21]:
from shotgun_csp.generator.utils import structure_reconstructor

structure_reconstructor?

Signature:
structure_reconstructor(
    structure_list,
    *,
    return_primitive=True,
    relax_with_ase: ase.calculators.calculator.Calculator = None,
    scaling_factor=3,
    n_jobs=-1,
    backend='loky',
)
Docstring:
Reconstructs a list of structures with optional relaxation using ASE and returns a DataFrame with structure information.

Parameters:
-----------
structure_list : list
    List of structures to be reconstructed. Each structure should be a dictionary containing 'lattice', 'species', and 'coords'.
return_primitive : bool, optional
    If True, returns the primitive standard structure. If False, returns the conventional standard structure. Default is True.
relax_with_ase : Calculator, optional
    ASE calculator for relaxing the structure. If None, no relaxation is performed. Default is None.
scaling_factor : int, optional
    Factor by which to scale the lattice volume before relaxation. Default is 3.
n_jobs : int, optional
    Number of jobs to run in parallel. Def

In [23]:
structure_reconstructor([raw_s], n_jobs=1)

,volume,formula,composition,reduced_formula,num_atoms,space_group,space_group_num,species,structure,min_max_dist,wy_letters,ratio,ratioSG,errors_msg
0,50.740424,Ca2 C2 O6,"{'C': 2.0, 'Ca': 2.0, 'O': 6.0}",CaCO3,10.0,R-3m,166,"[C, C, Ca, Ca, O, O, O, O, O, O]","{'@module': 'pymatgen.core.structure', '@class...","(2.6855117661853427, 1.8784206150462888)","[a, a, b, b, d, d, d, d, d, d]","(2.0, 2.0, 6.0)",2.0_2.0_6.0-166,None


The following is batched generation.

In [24]:
%%time

raw_ss = cg.gen_many(100, *cfgs)

print(f"raw_ss[0]: \n{raw_ss[0]},\nsize: {len(raw_ss)}")

raw_ss[0]: 
{'spacegroup_num': 167, 'volume': 153.232425242463, 'lattice': [[5.340705057646879, 0.0, 1.0614067912581733], [0.8712221057652152, 5.269165261709009, 1.0614067912581733], [0.0, 0.0, 5.445155175870006]], 'species': ['C', 'C', 'Ca', 'Ca', 'O', 'O', 'O', 'O', 'O', 'O'], 'wyckoff_letters': ['a', 'a', 'b', 'b', 'd', 'd', 'd', 'd', 'd', 'd'], 'coords': [[0.25, 0.25, 0.25], [0.75, 0.75, 0.75], [0.0, 0.0, 0.0], [0.5, 0.5, 0.5], [0.5, 0.0, 0.0], [0.0, 0.5, 0.0], [0.0, 0.0, 0.5], [0.5, 0.0, 0.5], [0.0, 0.5, 0.5], [0.5, 0.5, 0.0]]},
size: 95
CPU times: user 454 ms, sys: 47.9 ms, total: 502 ms
Wall time: 16.6 ms


Also, the iterative version

In [25]:
%%time

with tqdm(total=len(cfgs)) as pbar:
    for cfg, structures in cg.gen_many_iter(500, *cfgs):
        print(f"configuration: {cfg}, size of structures: {len(structures)}")
        pbar.update()

  0%|          | 0/2 [00:00<?, ?it/s]

configuration: {'C': ['a'], 'Ca': ['b'], 'O': ['d']}, size of structures: 382
configuration: {'C': ['b'], 'Ca': ['a'], 'O': ['d']}, size of structures: 66
CPU times: user 1.29 s, sys: 11.8 ms, total: 1.3 s
Wall time: 57.7 ms
